# Resume Optimizer

This interactive tool helps you optimize your resume for specific job applications by:
1. Selecting a job type with pre-configured keywords
2. Choosing a template optimized for your target role
3. Adding missing keywords to ensure ATS compatibility
4. Generating a professionally formatted PDF resume

## Instructions:

1. Complete the form below:
   - Enter your full name (first and last)
   - Select a job type (e.g., Data Engineer)
   - Choose a template variant (e.g., general, company-specific)
   - Set whether to upload the final PDF to Google Drive

2. Click the "Generate Resume" button to create your optimized resume.

3. Your files will be saved in the `data/output` directory.

In [ ]:
import os
import sys
import pickle
import tqdm.notebook as tqdm
import glob
import ipywidgets as widgets
from IPython.display import clear_output, display
from pathlib import Path

# Add scripts directory to path
sys.path.append('scripts')

# Import required modules
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from scripts.resume_generator import process_keywords, generate_resume, generate_pdf, upload_to_drive

# Get available job types and their templates
job_types = []
job_types_templates = {}

# Get all available job types
if os.path.exists('job_types'):
    job_types = [name for name in os.listdir('job_types') 
               if os.path.isdir(os.path.join('job_types', name))]
    
    # For each job type, get its templates
    for job_type in job_types:
        templates_dir = os.path.join('job_types', job_type, 'templates')
        if os.path.exists(templates_dir):
            templates = [os.path.splitext(name)[0] for name in os.listdir(templates_dir) 
                       if name.endswith('.md')]
            job_types_templates[job_type] = templates

# Create widgets for user input
name_input = widgets.Text(
    value='Malachi Dunn',
    description='Full Name:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

job_type_dropdown = widgets.Dropdown(
    options=job_types,
    value=job_types[0] if job_types else None,
    description='Job Type:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

template_dropdown = widgets.Dropdown(
    options=job_types_templates.get(job_types[0], []) if job_types else [],
    value=job_types_templates.get(job_types[0], [])[0] if job_types and job_types_templates.get(job_types[0], []) else None,
    description='Template:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

upload_dropdown = widgets.Dropdown(
    options=[('Yes', True), ('No', False), ('Skip PDF generation', 'skip')],
    value=True,
    description='Upload to Google Drive:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Static values
output_dir = "data/output"
drive_folder_id = "1s6n9eS9d2NNy9lyXgoYdPGOwkyGGujnw"

# Update template dropdown when job type changes
def update_templates(*args):
    template_dropdown.options = job_types_templates.get(job_type_dropdown.value, [])
    if template_dropdown.options:
        template_dropdown.value = template_dropdown.options[0]
    
job_type_dropdown.observe(update_templates, names='value')

# Create a button to start the process
run_button = widgets.Button(
    description='Generate Resume',
    button_style='success',
    icon='rocket'
)

output_area = widgets.Output()

def on_run_button_clicked(b):
    with output_area:
        clear_output()
        
        # Get values from widgets
        basename = name_input.value.strip()
        if not basename:
            print("❌ Error: Please enter a name.")
            return
            
        job_type = job_type_dropdown.value
        if not job_type:
            print("❌ Error: Please select a job type.")
            return
            
        template = template_dropdown.value
        if not template:
            print("❌ Error: Please select a template.")
            return
            
        do_upload = upload_dropdown.value
        
        # Setup progress bar
        total_steps = 2
        if do_upload != 'skip':
            total_steps += 1
        
        progress = tqdm.tqdm(total=total_steps, desc="Resume Generator")
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        # Set paths based on job type and template
        keywords_file = os.path.join('job_types', job_type, 'keywords.csv')
        template_file = os.path.join('job_types', job_type, 'templates', f"{template}.md")
        
        # Validate paths
        if not os.path.exists(keywords_file):
            print(f"❌ Error: Keywords file not found for job type '{job_type}'")
            return
            
        if not os.path.exists(template_file):
            print(f"❌ Error: Template file not found: {template_file}")
            return
        
        # Process keywords
        keywords_output = os.path.join(output_dir, 'processed_keywords.csv')
        keywords = process_keywords(keywords_file, keywords_output)
        progress.update(1)
        
        # Generate resume
        basename_underscore = basename.replace(' ', '_').lower()
        base_output = os.path.join(output_dir, f"{basename_underscore}_resume.md")
        generate_resume(template_file, None, base_output, keywords_file=keywords_output)
        progress.update(1)
        
        # Generate PDF and upload to Google Drive
        if do_upload != 'skip':
            pdf_file = generate_pdf(base_output, output_dir)
            if pdf_file:
                if do_upload is True:
                    drive_link = upload_to_drive(pdf_file, drive_folder_id)
                    upload_message = f"and uploaded to Google Drive"
                else:
                    upload_message = f"(not uploaded to Google Drive)"
                progress.update(1)
            else:
                upload_message = f"but PDF generation failed"
        else:
            upload_message = "(PDF generation skipped)"
        
        progress.close()
        print(f"✅ Done! Your resume has been generated {upload_message}.")
        print(f"📄 Output files are in: {output_dir}")

# Connect the button to the function
run_button.on_click(on_run_button_clicked)

# Display the widgets
display(widgets.HTML("<h3>Resume Generator Options</h3>"))
display(name_input)
display(job_type_dropdown)
display(template_dropdown)
display(upload_dropdown)
display(run_button)
display(output_area)